In [10]:
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def data_preprocessing():
  directory="/content/drive/MyDrive/classification/dataset/"
  categories=["with_mask","without_mask"]
  #lists where the values needs to be stored
  data=[]
  label=[]
  for category in categories:
      path=os.path.join(directory,category)
      for img in os.listdir(path):
          img_path=os.path.join(path,img)
          image=load_img(img_path,target_size=(224,224))
          image=img_to_array(image)
          image=preprocess_input(image)

          #appending the values    
          data.append(image)
          label.append(category)

  #encoding the label
  label=LabelEncoder().fit_transform(label)
  label=to_categorical(label)

  #converting the data and label into a numpy array
  data = np.array(data, dtype="float32")
  label = np.array(label) 

  #returning the lists
  return data,label

In [4]:
def image_augumentation():
  #constructing image data generator for image augumentation
  datagen=ImageDataGenerator(
      rotation_range=90,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
      )
  return datagen

In [5]:
def create_model():
  #constructing the base model using resnet50 with the head left off
  base_Model=ResNet50(weights="imagenet",include_top=False,input_tensor=Input(shape=(224, 224, 3)))
  #constructing the head of the model
  model=Flatten()(base_Model.output)
  model=Dense(100,activation="relu",kernel_initializer="he_uniform")(model)
  model=Dense(2,activation="softmax")(model)
  model=Model(inputs=base_Model.input, outputs=model)
  model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  return model

In [13]:
def run_system():
  #data
  data,label=data_preprocessing()
  #split the dataset
  X_train,X_test,y_train,y_test=train_test_split(data,label,test_size=0.20, random_state=42)
  #image augumentation
  datagen=image_augumentation()
  #get the created model
  model=create_model()
  #fit the model
  batch_size=32
  model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), 
                      epochs=9, # one forward/backward pass of training data
                      steps_per_epoch=X_train.shape[0]//batch_size, # number of images comprising of one epoch
                      validation_data=(X_test, y_test), # data for validation
                      validation_steps=X_test.shape[0]//batch_size)
  #evaluating performance
  result=model.evaluate(X_test,y_test)
  print("\n val loss:{}, val_accuracy:{}%".format(round(result[0],3),round(result[1]*100,3)))
  #saving the model
  model.save("face_model.h5")

In [7]:
#runs the entire code
run_system()

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/9
95/95 [==============================] - 117s 850ms/step - loss: 1.3240 - accuracy: 0.8138 - val_loss: 6.2753 - val_accuracy: 0.6806
Epoch 2/9
95/95 [==============================] - 75s 791ms/step - loss: 0.2627 - accuracy: 0.9084 - val_loss: 0.6289 - val_accuracy: 0.8879
Epoch 3/9
95/95 [==============================] - 75s 791ms/step - loss: 0.1680 - accuracy: 0.9476 - val_loss: 0.1915 - val_accuracy: 0.9400
Epoch 4/9
95/95 [==============================] - 75s 790ms/step - loss: 0.2016 - accuracy: 0.9331 - val_loss: 0.1703 - val_accuracy: 0.9635
Epoch 5/9
95/95 [==============================] - 76s 794ms/step - loss: 0.1834 - accuracy: 0.9426 - val_loss: 47.0258 - val_accuracy: 0.5398
Epoch 6/9
95/95 [==============================] - 75s 793ms/step - loss: 0.2210 - accuracy: 0.9314 - val_loss: 0.1054 - val_accuracy: 0.9648
Epoch 7/9
95/95 [==============================] - 76s 793ms/step - loss: 0.1695 - accuracy: 0.9446 - val_loss: 0.1300 - val_accuracy: 0.9622
Epoc

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
